# 第二次作业_02_2.8

> 新雅62/CDIE6 2016013327 项雨桐

## 0. 导入包和数据

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_table('HomeworkData/breast-cancer-wisconsin.txt',header=None)

## 1. 数据清洗

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
0     699 non-null int64
1     699 non-null int64
2     699 non-null int64
3     699 non-null int64
4     699 non-null int64
5     699 non-null int64
6     699 non-null object
7     699 non-null int64
8     699 non-null int64
9     699 non-null int64
10    699 non-null int64
dtypes: int64(10), object(1)
memory usage: 60.1+ KB


In [5]:
def cleaning(data):
    data = data.apply(pd.to_numeric, errors='coerce')
    data = data.dropna()
    data = data.astype('int64')
    for i in range(1,10):
        data = data[(data[i]<=10)&(data[i]>=1)]
    data = data[(data[10]==0)|(data[10]==1)]
    return data

data = cleaning(data)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
0     683 non-null int64
1     683 non-null int64
2     683 non-null int64
3     683 non-null int64
4     683 non-null int64
5     683 non-null int64
6     683 non-null int64
7     683 non-null int64
8     683 non-null int64
9     683 non-null int64
10    683 non-null int64
dtypes: int64(11)
memory usage: 64.0 KB


## 2. 数据准备

### 2.1 Logistic Regression

In [7]:
X = (data.iloc[:,1:10]).values
X_train = X[:500]
X_test = X[500:]

y = (data.iloc[:,10]).values
y_train = y[:500]
y_test = y[500:]

### 2.2 Fisher Linear Discriminant

In [8]:
data_train = data[:500]
data_test = data[500:]

data_benign = data_train[data_train[10]==0]
X_benign = (data_benign.iloc[:,1:10]).values

data_malign = data_train[data_train[10]==1]
X_malign = (data_malign.iloc[:,1:10]).values

## 3. 模型预测

### 3.1 Logistic Regression

In [9]:
model = LogisticRegressionCV(cv=5, random_state=0)
model.fit(X_train, y_train)

print(model.coef_,model.intercept_)
print(model.predict(X_test))
print(model.score(X_test,y_test))

[[ 0.52603063 -0.06659919  0.3236895   0.29804726  0.12426795  0.36230969
   0.35821194  0.19446375  0.46607639]] [-9.22746545]
[1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1
 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1]
1.0


### 3.2 Fisher's Linear Discriminant

In [10]:
#类均值向量
m1 = np.mean(X_benign, axis=0)
m2 = np.mean(X_malign, axis=0)

#类内离散度矩阵
S1 = np.zeros((m1.shape[0],m1.shape[0]))
S2 = np.zeros((m2.shape[0],m2.shape[0]))

for i,vec in enumerate((X_benign)):
    diff1 = (vec-m1).reshape(-1,1)
    S1 += np.dot(diff1,diff1.T)
    
for i,vec in enumerate((X_malign)):
    diff2 = (vec-m2).reshape(-1,1)
    S2 += np.dot(diff2,diff2.T)

Sw = S1 + S2

#分类面方向极值解
w = np.dot(np.linalg.inv(Sw), m1 - m2)
print(w)

#截距
w0 = -(np.dot(w,m1)+np.dot(w,m2))/2
print(w0)

[-0.00179037 -0.00055191 -0.00090057 -0.00040891 -0.00050946 -0.00227471
 -0.00055596 -0.00094394 -0.00023939]
0.0345932377178


In [12]:
y_pred = np.zeros(y_test.shape[0])

for i,vec in enumerate((X_test)):
    if np.dot(w,vec) + w0 > 0:
        y_pred[i] = 0
    else:
        y_pred[i] = 1

print(y_pred)
print(accuracy_score(y_test,y_pred))

[ 1.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.
  1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.
  0.  1.  0.  0.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.
  1.  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  1.  1.  1.]
0.994535519126
